In [1]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [2]:
!pip install opendatasets
!pip install pandas

Библиотеки

In [3]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

Скачиваем датасет  
Чтение датасета  
Приводим таргет к числовому типу, удаляя строки, которые не удалось конвертировать  
В House price of unit area есть цена 0, что может помешать модели, поэтому удаляем  

In [4]:
od.download("https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression")

pd.set_option("display.max_columns", None)
df = pd.read_csv("house-price-prediction-simplified-for-regression/Real_Estate.csv")

df["House price of unit area"] = pd.to_numeric(df["House price of unit area"], errors="coerce")
df = df.dropna(subset=["House price of unit area"])

df = df[df["House price of unit area"] > 0].copy()

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression


100%|██████████| 17.4k/17.4k [00:00<00:00, 60.7MB/s]

Формирую матрицу признаков для регрессии по ценам недвижимости и строю бейзлайновую модель GradientBoostingRegressor  
В качестве признаков беру возраст дома, расстояние до MRT, число магазинов, широту и долготу  
Числовые признаки стандартизую, далее обучаю бустинг с параметрами по умолчанию

Базовый градиентный бустинг без подбора гиперпараметров даёт MAE ≈ 10.55, RMSE ≈ 12.38 и R² ≈ 0.15. То есть модель уже что-то учит, но объясняет вариацию цен довольно слабо и ошибается в среднем на ~10 единиц цены за квадратный метр

In [5]:
features = [
    "House age",
    "Distance to the nearest MRT station",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X = df[features]
y = df["House price of unit area"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

preprocess_gb = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features),
    ]
)

gb_base = Pipeline(steps=[
    ("preprocess", preprocess_gb),
    ("model", GradientBoostingRegressor(random_state=42))
])

gb_base.fit(X_train, y_train)
y_pred_base = gb_base.predict(X_test)

mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2_base = r2_score(y_test, y_pred_base)

print("Бейзлайн")
print("MAE:", mae_base)
print("RMSE:", rmse_base)
print("R^2:", r2_base)

Бейзлайн
MAE: 10.552408093514218
RMSE: 12.376984270914132
R^2: 0.1530336711587641


Гипотеза 1 - подбор основных гиперпараметров: числа деревьев, скорости обучения и глубины базовых деревьев

После подбора гиперпараметров (более мелкие деревья и меньший learning_rate) MAE снижается до ≈ 10.09, RMSE до ≈ 11.76, R^2 растёт до ≈ 0.236. Ошибка чуть уменьшается, модель становится более сглаженной и стабильной, но прорыва по метрикам нет - это аккуратное, но не радикальное улучшение по сравнению с бейзлайном

In [6]:
param_grid_gb_1 = {
    "model__n_estimators": [100, 200, 300],
    "model__learning_rate": [0.05, 0.1, 0.2],
    "model__max_depth": [2, 3, 4]
}

gb_gs_1 = GridSearchCV(
    estimator=gb_base,
    param_grid=param_grid_gb_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

gb_gs_1.fit(X_train, y_train)

print("Лучшие параметры:", gb_gs_1.best_params_)
print("Лучший MAE на CV:", -gb_gs_1.best_score_)

best_gb_1 = gb_gs_1.best_estimator_
y_pred_1 = best_gb_1.predict(X_test)

mae_1 = mean_absolute_error(y_test, y_pred_1)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
r2_1 = r2_score(y_test, y_pred_1)

print("\nГипотеза 1")
print("MAE:", mae_1)
print("RMSE:", rmse_1)
print("R^2:", r2_1)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 9.993212187679118

Гипотеза 1
MAE: 10.090392425795702
RMSE: 11.75693830650001
R^2: 0.23576847996758077


Гипотеза 2 - расстояние до MRT имеет сильно скошенное распределение, поэтому вместо «сырых» расстояний попробую использовать логарифм расстояния

Логарифмирование расстояния до метро при тех же гиперпараметрах даёт те же самые значения метрик, что и гипотеза 1. Это означает, что для градиентного бустинга на этом датасете такая трансформация признака «Distance to MRT» по сути не добавляет информации: деревья и так умеют хорошо работать с исходными значениями и не выигрывают от log-преобразования

In [7]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocess_gb_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

gb_log = Pipeline(steps=[
    ("preprocess", preprocess_gb_log),
    ("model", GradientBoostingRegressor(random_state=42))
])

param_grid_gb_2 = param_grid_gb_1

gb_gs_2 = GridSearchCV(
    estimator=gb_log,
    param_grid=param_grid_gb_2,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

gb_gs_2.fit(X_train_log, y_train_log)

print("Лучшие параметры:", gb_gs_2.best_params_)
print("Лучший MAE на CV:", -gb_gs_2.best_score_)

best_gb_2 = gb_gs_2.best_estimator_
y_pred_2 = best_gb_2.predict(X_test_log)

mae_2 = mean_absolute_error(y_test_log, y_pred_2)
rmse_2 = np.sqrt(mean_squared_error(y_test_log, y_pred_2))
r2_2 = r2_score(y_test_log, y_pred_2)

print("\nGradientBoostingRegressor + log(расстояния до MRT)")
print("MAE:", mae_2)
print("RMSE:", rmse_2)
print("R^2:", r2_2)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 9.993212187679118

GradientBoostingRegressor + log(расстояния до MRT)
MAE: 10.090392425795702
RMSE: 11.75693830650001
R^2: 0.23576847996758077


3 гипотеза - логарифмирование таргета

Когда мы переходим к прогнозированию логарифма цены (а потом возвращаемся к исходной шкале), MAE падает до ≈ 9.87, RMSE — до ≈ 11.52, а R^2 растёт до ≈ 0.27. Логарифмирование таргета помогает сгладить влияние дорогих объектов и сделать распределение ошибок более равномерным, поэтому именно эта конфигурация даёт лучшую точность среди всех проверенных вариантов sklearn-модели

In [8]:
y_log_target = np.log1p(df["House price of unit area"])
X_base = df[features]

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_base, y_log_target,
    test_size=0.2,
    random_state=42
)

preprocess_gb_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features),
    ]
)

gb_log_y = Pipeline(steps=[
    ("preprocess", preprocess_gb_t),
    ("model", GradientBoostingRegressor(random_state=42))
])

param_grid_gb_3 = param_grid_gb_1

gb_gs_3 = GridSearchCV(
    estimator=gb_log_y,
    param_grid=param_grid_gb_3,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

gb_gs_3.fit(X_train_t, y_train_t)

print("Лучшие параметры:", gb_gs_3.best_params_)
print("Лучший MAE на CV:", -gb_gs_3.best_score_)

best_gb_3 = gb_gs_3.best_estimator_

y_pred_log = best_gb_3.predict(X_test_t)
y_pred_3 = np.expm1(y_pred_log)

y_test_true = np.expm1(y_test_t)

mae_3 = mean_absolute_error(y_test_true, y_pred_3)
rmse_3 = np.sqrt(mean_squared_error(y_test_true, y_pred_3))
r2_3 = r2_score(y_test_true, y_pred_3)

print("\nGradientBoostingRegressor + логарифм таргета")
print("MAE:", mae_3)
print("RMSE:", rmse_3)
print("R^2:", r2_3)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 0.363232610566802

GradientBoostingRegressor + логарифм таргета
MAE: 9.868580230321113
RMSE: 11.519322207002325
R^2: 0.26634763910905235


Реализую свои версии деревьев и градиентного бустинга для регрессии  
Сначала пишу MyDecisionTreeRegressor: это простое дерево решений, которое на каждом шаге перебирает признаки и пороги, ищет разбиение, минимизирующее среднеквадратичную ошибку (MSE), и в листьях хранит среднее значение таргета.
Затем делаю MyRandomForestRegressor как ансамбль из таких деревьев: для каждого дерева беру бутстрап-выборку объектов и случайное подмножество признаков, обучаю дерево и потом усредняю предсказания всех деревьев  
Наконец, реализую MyGradientBoostingRegressor  
1) начинаю с константного предсказания (среднее по y)
2) на каждой итерации считаю остатки y - current_pred и обучаю новое дерево на этих остатках
3) добавляю его вклад к текущему предсказанию с шагом learning_rate.
В итоге получаю свой градиентный бустинг по MSE, который по интерфейсу ведёт себя как обычный регрессионный алгоритм из sklearn и дальше используется в пайплайне с тем же препроцессингом, что и библиотечная модель

In [9]:
class MyDecisionTreeRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.n_features_ = n_features
        self.tree_ = []

        def mse(values):
            if values.size == 0:
                return 0.0
            mean = values.mean()
            return np.mean((values - mean) ** 2)

        def best_split(X_node, y_node, depth):
            n_samples_node, n_features_node = X_node.shape
            if n_samples_node < self.min_samples_split:
                return None, None, None, None
            if self.max_depth is not None and depth >= self.max_depth:
                return None, None, None, None

            best_feature = None
            best_threshold = None
            best_score = np.inf
            best_left_idx = None
            best_right_idx = None

            for feature in range(n_features_node):
                values = X_node[:, feature]
                thresholds = np.unique(values)
                for thr in thresholds:
                    left_idx = values <= thr
                    right_idx = values > thr
                    if left_idx.sum() < self.min_samples_leaf or right_idx.sum() < self.min_samples_leaf:
                        continue
                    mse_left = mse(y_node[left_idx])
                    mse_right = mse(y_node[right_idx])
                    score = (left_idx.sum() * mse_left + right_idx.sum() * mse_right) / n_samples_node
                    if score < best_score:
                        best_score = score
                        best_feature = feature
                        best_threshold = thr
                        best_left_idx = left_idx
                        best_right_idx = right_idx

            if best_feature is None:
                return None, None, None, None
            return best_feature, best_threshold, best_left_idx, best_right_idx

        def build_tree(X_node, y_node, depth):
            node = {}
            node["value"] = float(y_node.mean())

            feature, threshold, left_idx, right_idx = best_split(X_node, y_node, depth)
            if feature is None:
                node["feature"] = None
                node["threshold"] = None
                node["left"] = None
                node["right"] = None
                return node

            node["feature"] = feature
            node["threshold"] = threshold
            node["left"] = build_tree(X_node[left_idx], y_node[left_idx], depth + 1)
            node["right"] = build_tree(X_node[right_idx], y_node[right_idx], depth + 1)
            return node

        self.tree_ = build_tree(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        while node["feature"] is not None:
            if x[node["feature"]] <= node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["value"]

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        y_pred = np.zeros(n_samples, dtype=float)
        for i in range(n_samples):
            y_pred[i] = self._predict_one(X[i], self.tree_)
        return y_pred


class MyRandomForestRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.random_state = random_state

    def _get_n_features_to_sample(self, n_features):
        if isinstance(self.max_features, int):
            return min(self.max_features, n_features)
        if isinstance(self.max_features, float):
            return max(1, int(self.max_features * n_features))
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        return n_features

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.trees_ = []
        self.features_subsets_ = []

        n_features_sub = self._get_n_features_to_sample(n_features)

        for i in range(self.n_estimators):
            if self.bootstrap:
                indices = rng.randint(0, n_samples, size=n_samples)
            else:
                indices = np.arange(n_samples)

            feat_indices = rng.choice(n_features, size=n_features_sub, replace=False)

            X_boot = X[indices][:, feat_indices]
            y_boot = y[indices]

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=None
            )
            tree.fit(X_boot, y_boot)

            self.trees_.append(tree)
            self.features_subsets_.append(feat_indices)

        return self

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        preds = np.zeros((n_samples, len(self.trees_)), dtype=float)

        for i, (tree, feat_idx) in enumerate(zip(self.trees_, self.features_subsets_)):
            preds[:, i] = tree.predict(X[:, feat_idx])

        return preds.mean(axis=1)

class MyGradientBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y, dtype=float)

        self.init_ = np.mean(y)
        self.trees_ = []

        n_samples = X.shape[0]
        current_pred = np.full(n_samples, self.init_, dtype=float)

        rng = np.random.RandomState(self.random_state)

        for m in range(self.n_estimators):
            residuals = y - current_pred

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=None if self.random_state is None else self.random_state + m
            )

            tree.fit(X, residuals)
            self.trees_.append(tree)

            update = tree.predict(X)
            current_pred += self.learning_rate * update

        return self

    def predict(self, X):
        X = np.asarray(X)
        y_pred = np.full(X.shape[0], self.init_, dtype=float)

        for tree in self.trees_:
            y_pred += self.learning_rate * tree.predict(X)

        return y_pred

Бейзлайн

В базовой конфигурации своя реализация даёт MAE ≈ 10.62, RMSE ≈ 12.40 и R^2 ≈ 0.15. Это очень близко к библиотечному бейзлайну (чуть хуже по ошибкам), что уже показывает, что общий алгоритм реализован адекватно

In [10]:
features = [
    "House age",
    "Distance to the nearest MRT station",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X = df[features]
y = df["House price of unit area"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

preprocess_my_gb = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features),
    ]
)

my_gb_base = Pipeline(steps=[
    ("preprocess", preprocess_my_gb),
    ("model", MyGradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_gb_base.fit(X_train, y_train)
y_pred_my_base = my_gb_base.predict(X_test)

mae_my_base = mean_absolute_error(y_test, y_pred_my_base)
rmse_my_base = np.sqrt(mean_squared_error(y_test, y_pred_my_base))
r2_my_base = r2_score(y_test, y_pred_my_base)

print("Бейзлайн")
print("MAE:", mae_my_base)
print("RMSE:", rmse_my_base)
print("R^2:", r2_my_base)


Бейзлайн
MAE: 10.618062606022761
RMSE: 12.395913639515339
R^2: 0.15044098822927754


Гипотеза 1

После подбора n_estimators / max_depth / learning_rate MAE снижается до ≈ 10.08, RMSE — до ≈ 11.75, R^2 поднимается до ≈ 0.237. Поведение почти полностью совпадает со sklearn-версией: аккуратный выигрыш по качеству за счёт более удачных настроек, без кардинального скачка

In [11]:
param_grid_my_gb_1 = {
    "model__n_estimators": [100, 200, 300],
    "model__learning_rate": [0.05, 0.1, 0.2],
    "model__max_depth": [2, 3, 4]
}

my_gb_gs_1 = GridSearchCV(
    estimator=my_gb_base,
    param_grid=param_grid_my_gb_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_gb_gs_1.fit(X_train, y_train)

print("Лучшие параметры:", my_gb_gs_1.best_params_)
print("Лучший MAE на CV:", -my_gb_gs_1.best_score_)

best_my_gb_1 = my_gb_gs_1.best_estimator_
y_pred_my_1 = best_my_gb_1.predict(X_test)

mae_my_1 = mean_absolute_error(y_test, y_pred_my_1)
rmse_my_1 = np.sqrt(mean_squared_error(y_test, y_pred_my_1))
r2_my_1 = r2_score(y_test, y_pred_my_1)

print("\nГипотеза 1")
print("MAE:", mae_my_1)
print("RMSE:", rmse_my_1)
print("R^2:", r2_my_1)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 10.065510997281969

Гипотеза 1
MAE: 10.080777640697098
RMSE: 11.746663273588833
R^2: 0.23710370389214264


Гипотеза 2

Логарифмирование расстояния до MRT в собственной реализации снова даёт идентичные метрики гипотезе 1. Значит, как и в случае со sklearn-моделью, эта трансформация для бустинга на данном датасете не играет заметной роли, модель уже умеет «переваривать» исходный масштаб признака

In [12]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocess_my_gb_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

my_gb_log = Pipeline(steps=[
    ("preprocess", preprocess_my_gb_log),
    ("model", MyGradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

param_grid_my_gb_2 = param_grid_my_gb_1

my_gb_gs_2 = GridSearchCV(
    estimator=my_gb_log,
    param_grid=param_grid_my_gb_2,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_gb_gs_2.fit(X_train_log, y_train_log)

print("Лучшие параметры:", my_gb_gs_2.best_params_)
print("Лучший MAE на CV:", -my_gb_gs_2.best_score_)

best_my_gb_2 = my_gb_gs_2.best_estimator_
y_pred_my_2 = best_my_gb_2.predict(X_test_log)

mae_my_2 = mean_absolute_error(y_test_log, y_pred_my_2)
rmse_my_2 = np.sqrt(mean_squared_error(y_test_log, y_pred_my_2))
r2_my_2 = r2_score(y_test_log, y_pred_my_2)

print("\nMyGradientBoostingRegressor + log(расстояния до MRT)")
print("MAE:", mae_my_2)
print("RMSE:", rmse_my_2)
print("R^2:", r2_my_2)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 10.065510997281969

MyGradientBoostingRegressor + log(расстояния до MRT)
MAE: 10.080777640697098
RMSE: 11.746663273588833
R^2: 0.23710370389214264


Гипотеза 3

При работе с логарифмом таргета MAE снижается до ≈ 9.75, RMSE — до ≈ 11.46, а R^2 растёт до ≈ 0.274. Это лучший вариант среди всех конфигураций собственной реализации и по динамике метрик он очень близок к библиотечному бустингу. То есть логарифмирование целевой переменной снова оказывается наиболее полезным приёмом, а совпадение поведения с sklearn говорит, что MyGradientBoostingRegressor реализован корректно и воспроизводит ту же логику обучения, что и стандартная библиотечная модель

In [13]:
y_log_target = np.log1p(df["House price of unit area"])
X_base = df[features]

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_base, y_log_target,
    test_size=0.2,
    random_state=42
)

preprocess_my_gb_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features),
    ]
)

my_gb_log_y = Pipeline(steps=[
    ("preprocess", preprocess_my_gb_t),
    ("model", MyGradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

param_grid_my_gb_3 = param_grid_my_gb_1

my_gb_gs_3 = GridSearchCV(
    estimator=my_gb_log_y,
    param_grid=param_grid_my_gb_3,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_gb_gs_3.fit(X_train_t, y_train_t)

print("Лучшие параметры:", my_gb_gs_3.best_params_)
print("Лучший MAE на CV:", -my_gb_gs_3.best_score_)

best_my_gb_3 = my_gb_gs_3.best_estimator_

y_pred_log_my = best_my_gb_3.predict(X_test_t)
y_pred_my_3 = np.expm1(y_pred_log_my)

y_test_true = np.expm1(y_test_t)

mae_my_3 = mean_absolute_error(y_test_true, y_pred_my_3)
rmse_my_3 = np.sqrt(mean_squared_error(y_test_true, y_pred_my_3))
r2_my_3 = r2_score(y_test_true, y_pred_my_3)

print("\nMyGradientBoostingRegressor + логарифм таргета")
print("MAE:", mae_my_3)
print("RMSE:", rmse_my_3)
print("R^2:", r2_my_3)


Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 0.3691941098657465

MyGradientBoostingRegressor + логарифм таргета
MAE: 9.746978165936188
RMSE: 11.460055094202803
R^2: 0.2738775267227396


Бейзлайны у sklearn и моей реализации дают MAE ≈ 10–10.6 и R² около 0.15, после подбора гиперпараметров качество немного улучшается, но не радикально. Логарифм расстояния до MRT почти ничего не даёт, а вот логарифмирование таргета даёт заметный выигрыш: MAE падает до ~9.7–9.9, R^2 растёт до ~0.27. При этом моя реализация ведёт себя очень похоже на библиотечный GradientBoostingRegressor, так что её можно считать корректной — ключевую роль в улучшении играет не алгоритм как таковой, а грамотная работа с таргетом